# Import Data

In [1]:
"""Import Packages for Plot, Numpy, OS dir, Copy, Math, Video Processing and I kept Matplolib to qt so that it will 
generate external figures  for multiple images rather than reduce the size and display internal to the jupyter notebook"""

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from collections import defaultdict
import numpy as np
import cv2
import os
import copy
import math
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib qt

print ('Importing done')

Importing done


# Function Definitions

In [4]:
#Grayscale Funtion
def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
#Canny Transform
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

#Gaussian Blur
def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

#Region of Interest mask
def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

"""Important Modified Draw Lines Function"""
def draw_lines(img, lines, color=[0,0,255], thickness=5):
    global Frame
    global cache
    global Stack_Counter
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    
    #global Centre Point
    global Image_Centre_Point
    
    #try block incase the Frame did not find any lanes to plot. Except would read from old cache until new lanes found
    try:
        
        """Create Lines array to store the following
        1) Points in lines
        2) Slopes
        3) Midpoints
        4) Distance from centre of Image
        5) Direction/Location of line in image (Right vs left)
        6) Avg of all points on each side      """
        
        lines_Array = defaultdict(list)
        
        #Calculate Slopes and Mid
        for line in lines:
            for x1,y1,x2,y2 in line:
                lines_Array['points'].append([x1,y1,x2,y2])
                lines_Array['slope'].append((y2-y1)/(x2-x1))
                lines_Array['midpoint'].append([(x1+x2)/2,(y1+y2)/2])

        # Find the total number of lanes present
        for key,value in lines_Array.items():
            Line_Count=sum(1 for v in value if v)

        # Find if lane is left lane or right lane based on positive or negative slope
        for i in range(0,Line_Count):
            if lines_Array['slope'][i] >0:
                lines_Array['direction'].append('right')

            if lines_Array['slope'][i] <0:
                lines_Array['direction'].append('left')
        
        #Create Empty Array to store all data of Left and Right Lanes.
        
        left_lanes=defaultdict(list)
        right_lanes=defaultdict(list)

        #Classification of Left or Right lines.
        for i in range(0,Line_Count-1):
            #print('index',i)
            #print('length',len(lines_Array['direction']))
            if lines_Array['direction'][i] == 'left':
                left_lanes['points'].append(lines_Array['points'][i])
                left_lanes['slope'].append(lines_Array['slope'][i])
                left_lanes['midpoint'].append(lines_Array['midpoint'][i])
                left_lanes['dist_from_centre'].append(dist_frm_Centre(lines_Array['midpoint'][i]))

        for i in range(0,Line_Count-1):
            if lines_Array['direction'][i] == 'right':
                right_lanes['points'].append(lines_Array['points'][i])
                right_lanes['slope'].append(lines_Array['slope'][i])
                right_lanes['midpoint'].append(lines_Array['midpoint'][i])
                right_lanes['dist_from_centre'].append(dist_frm_Centre(lines_Array['midpoint'][i]))
        
        #Calculate total number of left and right lines
        
        left_Line_Count=0
        right_Line_Count=0
        for key,value in left_lanes.items():
            left_Line_Count=sum(1 for v in value if v)  
        for key,value in right_lanes.items():
            right_Line_Count=sum(1 for v in value if v)  

        """Two steps being done below:
        1) Assign Weights to lanes based on distance of line midpoint from Centre of Image
        2) Calculate Weighted Average of points based on assigned weights."""
        
        if left_Line_Count>0:
            left_assigned = assign_weights(left_lanes)
            lwa=weighted_average(left_assigned)
        else:
            return 
        if right_Line_Count>0:
            right_assigned = assign_weights(right_lanes)
            rwa=weighted_average(right_assigned)
        else:
            return

        # Create stach of Cache of 25 frames. 
        if Frame <=25:
            cache['lwa'+str(Frame)].append(lwa)
            cache['rwa'+str(Frame)].append(rwa)
        
        # If Frames exceed 25, remove top of stack and reinsert new value into stack.
        if Frame >25:
            cache.pop('lwa'+str(Frame-7))
            cache['lwa'+str(Frame)].append(lwa)
            cache.pop('rwa'+str(Frame-7))
            cache['rwa'+str(Frame)].append(rwa)
            Stack_Counter=Stack_Counter+1
        
        #Caluclate Average of Cache
                      
        new_lwa, new_rwa =Avg_cache(cache)

        Frame=Frame+1
        
        # Extrapolate third point close to bottom edge of immage based on new two points and then plot Line
                      
        third_point=((540-new_lwa[0][1])/((new_lwa[0][3]-new_lwa[0][1])/(new_lwa[0][2]-new_lwa[0][0])))+new_lwa[0][0]
        cv2.line(img, (int(new_lwa[0][2]), int(new_lwa[0][3])), (int(third_point), 540 ), color, thickness)
        third_point=((540-new_rwa[0][1])/((new_rwa[0][3]-new_rwa[0][1])/(new_rwa[0][2]-new_rwa[0][0])))+new_rwa[0][0]
        cv2.line(img, (int(third_point), 540),(int(new_rwa[0][0]), int(new_rwa[0][1])), color, thickness)
        
    except:
        print('no lane detected')
        #Caluclate Average of Cache
                      
        new_lwa, new_rwa =Avg_cache(cache)

        # Extrapolate third point close to bottom edge of immage based on new two points and then plot Line
                      
        third_point=((540-new_lwa[0][1])/((new_lwa[0][3]-new_lwa[0][1])/(new_lwa[0][2]-new_lwa[0][0])))+new_lwa[0][0]
        cv2.line(img, (int(new_lwa[0][2]), int(new_lwa[0][3])), (int(third_point), 540 ), color, thickness)
        third_point=((540-new_rwa[0][1])/((new_rwa[0][3]-new_rwa[0][1])/(new_rwa[0][2]-new_rwa[0][0])))+new_rwa[0][0]
        cv2.line(img, (int(third_point), 540),(int(new_rwa[0][0]), int(new_rwa[0][1])), color, thickness)
        return
    
                      
# Calculate average of Cache
def Avg_cache(cache):
    avg_cache=[]
    for key, value in cache.items():
        if 'lwa' in key:
            if value[0]['avg'] != [[]]:
                avg_cache.append(value[0]['avg'])
            
    lwa_avg = np.mean(avg_cache,axis=0)
    avg_cache=[]
    for key, value in cache.items():
        if 'rwa' in key:
            avg_cache.append(value[0]['avg'])
    rwa_avg = np.mean(avg_cache,axis=0)
    
    return lwa_avg, rwa_avg
    
#Calculate Weighted Average            
def weighted_average(lines):
    for key,value in lines.items():
        Line_Count=sum(1 for v in value if v)
    avg=[]   
    for i in range(0,Line_Count):
        a=copy.copy(np.array(lines['points'][i]))
        b=copy.copy(lines['weights'][i])
        avg.append(np.multiply(a,b))
    lines['avg'].append([sum(i) for i in zip(*avg)])
    return(lines)
    
# Create weights and Assign to points giving importance to points closer to centre of image.
def assign_weights(lines):
    #get the total number of midpoints to work with
    for key,value in lines.items():
        Line_Count=sum(1 for v in value if v)
    #Get assignable weights summed to 1 
    
    weights = np.sort(np.random.rand(1,Line_Count))
    weight_sum=np.sum(weights)
    for i in range(0,Line_Count):
        weights[0][i] = (weights[0][i]/weight_sum)
    weights=np.fliplr(weights)

    #assign weights
    lines['weights']=copy.copy(lines['dist_from_centre'])
    sorted_dist=np.sort(lines['dist_from_centre'])
    counter=0; # for weight iteration
    for i in sorted_dist:
        ind=lines['dist_from_centre'].index(i)
        #print('weights',(weights), 'counter',counter)
        lines['weights'][ind]=weights[0][counter]
        counter=counter+1
    return(lines)
        
# calculate Distance of midpoint of line from Centre of image.
def dist_frm_Centre(points):
    dist = np.sqrt(np.square(Image_Centre_Point[0]-points[0])+np.square(Image_Centre_Point[1]-points[1]))
    return dist
    
#hough Transform
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

# MAIN LOOP
def process_image(Image,display=False):
    
    global Image_Centre_Point
    global Frame
    global cache
    
    Image_Centre_Point=[Image.shape[1]/2,Image.shape[0]/2]
        
    #Convert to Grayscale
    Image_Gray = grayscale(Image)

    #Convert to HSV, extract yellow and White lines and Apply Mask
    Image_HSV = cv2.cvtColor(Image, cv2.COLOR_RGB2HSV)
    Image_Yellow =cv2.inRange(Image_HSV, np.array([20,75,100],dtype ='uint8'), np.array([30,255,255],dtype ='uint8'))
    Image_White =cv2.inRange(Image_HSV, np.array([0,0,150],dtype ='uint8'), np.array([255,255,255],dtype ='uint8'))
    Final_mask=cv2.bitwise_or(Image_Yellow,Image_White)

    #Apply Gausian Blur
    gaus_img2 =gaussian_blur(Final_mask,5)
    
    #Apply Canny Transform
    Canny_Image = canny(gaus_img2,100,150)

    #Apply Region Of Interest Mask
    cropped_Canny = region_of_interest(Canny_Image,np.array( [[[150,Image.shape[0]],[260,400],[500,310],[726 ,400],[850,Image.shape[0]]]], dtype=np.int32 ))

    # Hough Lines
    rho = 4
    theta = np.pi/180  *0.25
    threshold = 20
    min_line_length =100
    max_line_gap =75
    line_image = np.copy(Image)*0
    hough_img =hough_lines(cropped_Canny,rho,theta,threshold,min_line_length,max_line_gap)
    color_edges = np.dstack((Canny_Image, Canny_Image, Canny_Image)) 
    final_image= weighted_img(hough_img,Image)
    
    # Chose to display images or not.
    if display==True:
        fig1=plt.figure( figsize=(16,9))
        ax1=fig1.add_subplot(2,2,1)
        ax1=plt.imshow(Image)
        ax2=fig1.add_subplot(2,2,4)
        ax2=plt.imshow(final_image)
        ax3=fig1.add_subplot(2,2,2)
        ax3=plt.imshow(hough_img)
        ax4=fig1.add_subplot(2,2,3)
        ax4=plt.imshow(Canny_Image)
        fig1.show()
    
    return final_image

#Same function as above just with a modified White plane.
def process_image_Challenge(Image,display=False):
    
    global Image_Centre_Point
    global Frame
    global cache
    
    Image_Centre_Point=[Image.shape[1]/2,Image.shape[0]/2]
        
    #Convert to Grayscale
    Image_Gray = grayscale(Image)

    #Convert to HSV
    Image_HSV = cv2.cvtColor(Image, cv2.COLOR_RGB2HSV)
    Image_Yellow =cv2.inRange(Image_HSV, np.array([20,75,100],dtype ='uint8'), np.array([30,255,255],dtype ='uint8'))
    Image_White =cv2.inRange(Image, np.array([0,0,150],dtype ='uint8'), np.array([255,255,255],dtype ='uint8'))
    Final_mask=cv2.bitwise_or(Image_Yellow,Image_White)

    
    gaus_img2 =gaussian_blur(Final_mask,5)

    Canny_Image = canny(gaus_img2,80,200)

    cropped_Canny = region_of_interest(Canny_Image,np.array( [[[Image.shape[1]/9,Image.shape[0]],[Image.shape[1]/6,400],[500,300],[Image.shape[1]-Image.shape[1]/6 ,400],[Image.shape[1]-Image.shape[1]/9,Image.shape[0]]]], dtype=np.int32 ))

    #Hough Lines
    rho = 3
    theta = np.pi/180  *0.75
    threshold = 20
    min_line_length =100
    max_line_gap =75
    line_image = np.copy(Image)*0
    hough_img =hough_lines(cropped_Canny,rho,theta,threshold,min_line_length,max_line_gap)
    #plt.imshow(hough_img)
    color_edges = np.dstack((Canny_Image, Canny_Image, Canny_Image)) 
    
    final_image= weighted_img(hough_img,Image)
    
    if display==True:
        fig1=plt.figure( figsize=(16,9))
        ax1=fig1.add_subplot(2,2,1)
        ax1=plt.imshow(Image)
        ax2=fig1.add_subplot(2,2,4)
        ax2=plt.imshow(final_image)
        ax3=fig1.add_subplot(2,2,2)
        ax3=plt.imshow(hough_img)
        ax4=fig1.add_subplot(2,2,3)
        ax4=plt.imshow(Canny_Image)
        fig1.show()
        
	
    return final_image
        
print ('function initialization done')

function initialization done


# Main Code

**To Test All Images**

In [ ]:
global Frame
global cache
global Stack_Counter
Stack_Counter=1
cache=defaultdict(list)
Frame=1
for picture in os.listdir('test_images/'):
    Img = mpimg.imread('test_images/'+ picture)
    processed=process_image(Img,True)
    mpimg.imsave("processed_images/annotated_"+picture,processed)

**To Test Individual Images**

In [ ]:
global Frame
global cache
cache=defaultdict(list)
Frame=1
Img = mpimg.imread('test_images/solidWhiteRight.jpg')
Hough_i2=process_image(Img, True)



**To Test Videos**

Video 1

In [5]:
global Frame
global cache
global Stack_Counter
Stack_Counter=1
cache=defaultdict(list)
Frame=1
white_output = 'test_videos_output/solidWhiteRight.mp4'
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4


100%|███████████████████████████████████████▊| 221/222 [00:04<00:00, 51.00it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 

Wall time: 4.7 s


In [6]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

Video 2

In [7]:
global Frame
global cache
global Stack_Counter
Stack_Counter=1
cache=defaultdict(list)
Frame=1
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/solidYellowLeft.mp4


100%|███████████████████████████████████████▉| 681/682 [00:13<00:00, 48.77it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidYellowLeft.mp4 

Wall time: 14.3 s


In [8]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

Video 3

In [9]:
global Frame
global cache
global Stack_Counter
Stack_Counter=1
cache=defaultdict(list)
Frame=1
challenge_output = 'test_videos_output/challenge.mp4'
clip3 = VideoFileClip('test_videos/challenge.mp4')
new_clip= clip3.resize(height=540,width=96)
challenge_clip =new_clip.fl_image(process_image_Challenge)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/challenge.mp4
[MoviePy] Writing video test_videos_output/challenge.mp4


 13%|█████▍                                   | 33/251 [00:00<00:05, 42.05it/s]

no lane detected
no lane detected


 15%|██████                                   | 37/251 [00:00<00:05, 40.78it/s]

no lane detected
no lane detected


...

no lane detected
no lane detected
no lane detected


 57%|██████████████████████▉                 | 144/251 [00:04<00:04, 24.19it/s]

no lane detected
no lane detected
no lane detected


100%|████████████████████████████████████████| 251/251 [00:07<00:00, 32.43it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/challenge.mp4 

Wall time: 8.12 s


In [10]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))